<a href="https://colab.research.google.com/github/HazSyl1/GPT-From-Scratch/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-06 12:52:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-02-06 12:52:23 (21.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt') as f:
  text=f.read()

In [3]:
print(f"Length of file:{len(text):,}")

Length of file:1,115,394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#Tokenizing . Character lvl
st_to_i={ch:i for i,ch in enumerate(chars)}
i_to_st={i:ch for i,ch in enumerate(chars)}
enc=lambda s: [st_to_i[c] for c in s]
dec=lambda l:''.join([i_to_st[i] for i in l])

print("hii there")
print("Encoding:",enc("hii there"))
print("Decoder:",dec(enc("hii there")))

hii there
Encoding: [46, 47, 47, 1, 58, 46, 43, 56, 43]
Decoder: hii there


In [7]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [8]:
##TIKTOKEN
# !pip install tiktoken
import tiktoken
en=tiktoken.get_encoding('gpt2')
x=en.encode("hii there")
y=en.decode(x)
print("Encoded:",x)
print("Decoded:",y)
print("Vocab Len",en.n_vocab)

Encoded: [71, 4178, 612]
Decoded: hii there
Vocab Len 50257


In [9]:
#encode entire text in a tensor
import torch
data=torch.tensor(enc(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data.size())
print(data[:1000])

torch.Size([1115394]) torch.int64
torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 

In [10]:
#Spliting
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f"When input is {context} , the target is {target}")

When input is tensor([18]) , the target is 47
When input is tensor([18, 47]) , the target is 56
When input is tensor([18, 47, 56]) , the target is 57
When input is tensor([18, 47, 56, 57]) , the target is 58
When input is tensor([18, 47, 56, 57, 58]) , the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) , the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) , the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) , the target is 58


In [13]:
torch.manual_seed(1337)
batch_size=4#how many independant sequences will be processed in parallel...
block_size=8#what  is the max context length for predictions....

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,)) #batch_size number of random offsets b/w 0 to len(data)-block_size
  #print(f"{ix}:\n")
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:1+i+block_size] for i in ix])
  return x,y

xb,yb=get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb ,"\n")
print("Targets:")
print(yb.shape)
print(yb)
print("___________")

for b in range(batch_size): #batch dim
  for t in range(block_size):#time dim
    context=xb[b, :t+1]
    target=yb[b,t]
    print(f"When input is: {context} , target is : {target}")


Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 

Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
___________
When input is: tensor([24]) , target is : 43
When input is: tensor([24, 43]) , target is : 58
When input is: tensor([24, 43, 58]) , target is : 5
When input is: tensor([24, 43, 58,  5]) , target is : 57
When input is: tensor([24, 43, 58,  5, 57]) , target is : 1
When input is: tensor([24, 43, 58,  5, 57,  1]) , target is : 46
When input is: tensor([24, 43, 58,  5, 57,  1, 46]) , target is : 43
When input is: tensor([24, 43, 58,  5, 57,  1, 46, 43]) , target is : 39
When input is: tensor([44]) , target is : 53
When input is: tensor([44, 53]) , target is : 56
When input is: tensor([44, 53

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self ,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits=self.token_embedding_table(idx) #(B,T,C: Batch , Time , Channel)(4,8,65[vocab_size])
    #BUT pytorch expects , B ,C,T . so we re shape
    if targets ==None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C)
      targets=targets.view(B*T) #or -1 , cause pytorch can figure that by itself
      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    #idx is (B,T)
    for _ in range(max_new_tokens):
      logits,loss=self(idx)
      logits=logits[:,-1,:] #becomes (B,C) #Pluck out the last element int he TIme dimension,because those are the prediction of what comes next
      #print("LL",logits.shape)
      probs=F.softmax(logits,dim=-1) #(B,C)

      idx_next=torch.multinomial(probs,num_samples=1) #(B,1)
      idx=torch.cat((idx,idx_next),dim=1)#(B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

print(dec(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [15]:
optimizer=torch.optim.Adam(m.parameters(),lr=1e-3)

In [16]:
batch_size=32
for steps in range(10000):
  xb,yb=get_batch('train')

  logits,loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.559103012084961


In [17]:
print(dec(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=400)[0].tolist()))


Ong h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henourzincenonthioneir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprrouthercckehathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so itJas
Waketancothanan hay.JUCLUKILe, bs, r loncave w hollular s O:
HIs; ht anjx?

DUThineent.

LaZEESTEORDY:
h l.
KEONGBUCHandspo be y,-JZNEEYowddy scace, tridesar, wne'shenous s 


## Math Trick in self-attention

In [18]:
torch.manual_seed(1337)
B,T,C =4,8,2
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [20]:
#Info should only transfer from rpevious time stamp to current , casuse we're not aware of teh future lol
#Say i'm at C=5 , I'd like to avg the 4,3,2,1 ,to get an average.
#Now getting an avg is not really workfull , its shit , cause info gets lost. . . .
#For every T token we'd like to ge the avg of all before it . . .  .

In [35]:
##VERSION1
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t,C)
    xbow[b,t]=torch.mean(xprev,0) #Avereging the t


In [32]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [33]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [38]:
### VERSION2
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei @ x #(T,T) @ (B,T,C) -----> (B,T,T) @ (B,T,C) -----> (B,T,C)
torch.allclose(xbow,xbow2)

True

In [39]:
xbow[0] ,xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
#We were able to use matrix multiplication , to get the weighted sums . . .

In [43]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [48]:
wei.shape

torch.Size([8, 8])

In [51]:
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril == 0,float('-inf'))
wei=F.softmax(wei,dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [53]:
## VERSION 3 : Use SOFTMAX
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)
xbow3=wei @ x
xbow3[0]


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [26]:
 torch.manual_seed(42)
a=torch.tril(torch.ones(2,3))
a=a/torch.sum(a,1,keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
print('a=')
print(a)
print('b=')
print(b)
print('\nc=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000]])
